## Byte Pair Encoder (BPE)

In [1]:
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version('tiktoken'))

tiktoken version: 0.8.0


In [2]:
tokenizer = tiktoken.get_encoding("gpt2")

In [3]:
text = ("Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace")
integers = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
integers

[15496,
 11,
 466,
 345,
 588,
 8887,
 30,
 220,
 50256,
 554,
 262,
 4252,
 18250,
 8812,
 2114,
 286,
 617,
 34680,
 27271]

In [4]:
tokenizer.decode([345])

' you'

In [5]:
integers = tokenizer.encode("Akwirw ier")
print(integers)

strings = tokenizer.decode(integers)
print(strings)

[33901, 86, 343, 86, 220, 959]
Akwirw ier


In [6]:
tokenizer.decode([959])

'ier'

In [7]:
import tiktoken

# Initialize the encodings for GPT-2, GPT-3, and GPT-4
encodings = {
    "gpt2": tiktoken.get_encoding("gpt2"),
    "gpt3": tiktoken.get_encoding("p50k_base"), # Commonly associated with GPT-3 models
    "gpt4": tiktoken.get_encoding("cl100k_base") # Used for GPT-4 and later versions
}

# Get the vocabulary size for each encoding
vocab_sizes = {model: encoding.n_vocab for model, encoding in encodings.items()}

# Print the vocabulary sizes
for model, size in vocab_sizes.items():
    print(f"The vocabulary size for {model.upper()} is: {size:,}")

The vocabulary size for GPT2 is: 50,257
The vocabulary size for GPT3 is: 50,281
The vocabulary size for GPT4 is: 100,277


## Creating Input-target Pairs

In [8]:
# Open the file in read mode
with open('verdict.txt', 'r', encoding='utf-8') as file:
    # Read the contents of the file
    raw_text = file.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [9]:
enc_sample = enc_text[50:]

In [10]:
context_size = 4
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1] 

print(f"x: {x}")
print(f"y: {y}")

x: [290, 4920, 2241, 287]
y: [4920, 2241, 287, 257]


In [11]:
# Processing the inputs along with the targets, which are the inputs shifted by one position. We can then
# create the next-word prediction tasks as follows:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "---------->", desired)

[290] ----------> 4920
[290, 4920] ----------> 2241
[290, 4920, 2241] ----------> 287
[290, 4920, 2241, 287] ----------> 257


In [12]:
# For illustration purposes, let's repeat the previous code but convert the token IDs into text
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context), "---------->", tokenizer.decode([desired]))

 and ---------->  established
 and established ---------->  himself
 and established himself ---------->  in
 and established himself in ---------->  a


In [13]:
# Step1: Tokenize the entire text
# Step2: Use a sliding window to chunk the book into overlapping sequences of max_length
# Step3: Return the total number of rows in the dataset
# Step4: Return a single row from the dataset

In [14]:
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={'<|endoftext|>'})
        
        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i: i+max_length]
            target_chunk = token_ids[i+1: i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [15]:
# Step1: Initialize the tokenizer
# Step2: Create dataset
# Step3: drop_last = True drops the last batch if it is shorter than the specified batch_size to prevent loss spikes during training
# Step4: The number of CPU processes to use for preprocessing

In [16]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True, num_workers=0):
    
    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")
    
    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last, num_workers=num_workers)

    return dataloader

In [17]:
# Open the file in read mode
with open('verdict.txt', 'r', encoding='utf-8') as file:
    # Read the contents of the file
    raw_text = file.read()

In [18]:
# Let's test the dataloader with a batch size of 1 for an LLM with a context size of 4

# Convert dataloader into a python iterator to fetch the next entry

In [19]:
import torch
print("PyTorch version:", torch.__version__)

dataloader = create_dataloader_v1(raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

PyTorch version: 2.5.1+cu124
[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [21]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[2885, 1464, 1807, 3619]]), tensor([[1464, 1807, 3619,  402]])]


In [22]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])
